<a href="https://colab.research.google.com/github/shivaZeymaran/TRFPre-Code-Reviewer-Recommender/blob/main/TRFPre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Crawling Data

In [ ]:
!pip3 install PyGithub requests

     |████████████████████████████████| 291 kB 33.4 MB/s 
     |████████████████████████████████| 961 kB 39.2 MB/s 


In [ ]:
from github import Github

# authenticate to github
g = Github()

repo = g.search_repositories("angular/angular.js")[0]

print("Full name:", repo.full_name)
print("Description:", repo.description)
print("Date created:", repo.created_at)
print("Date of last push:", repo.pushed_at)
print("Home Page:", repo.homepage)
print("Language:", repo.language)
print("Number of forks:", repo.forks)
print("Number of stars:", repo.stargazers_count)
print("-"*50)
# repository content (files & directories)
print("Contents:")
for content in repo.get_contents(""):
    print(content)

Full name: angular/angular.js
Description: AngularJS - HTML enhanced for web apps!
Date created: 2010-01-06 00:34:37
Date of last push: 2021-12-09 23:20:01
Home Page: https://angularjs.org
Language: JavaScript
Number of forks: 28409
Number of stars: 59577
--------------------------------------------------
Contents:
ContentFile(path=".circleci")
ContentFile(path=".editorconfig")
ContentFile(path=".eslintignore")
ContentFile(path=".eslintrc-base.json")
ContentFile(path=".eslintrc-browser.json")
ContentFile(path=".eslintrc-node.json")
ContentFile(path=".eslintrc-todo.json")
ContentFile(path=".eslintrc.json")
ContentFile(path=".gitattributes")
ContentFile(path=".github")
ContentFile(path=".gitignore")
ContentFile(path=".mailmap")
ContentFile(path=".nvmrc")
ContentFile(path="CHANGELOG.md")
ContentFile(path="CODE_OF_CONDUCT.md")
ContentFile(path="CONTRIBUTING.md")
ContentFile(path="DEVELOPERS.md")
ContentFile(path="Gruntfile.js")
ContentFile(path="LICENSE")
ContentFile(path="README.closure.m

In [ ]:
import json

pulls = repo.get_pulls(state='closed', sort='created', direction='asc')

with open('output.json', 'a') as outfile:
  for pr in pulls:
    outfile.write(str(pr))

TypeError: ignored

In [ ]:
usernames_involved = set()

pr = pulls[100]

for review in pr.get_reviews():
    usernames_involved.add(review.user.username)

users_requested, teams_requested = pr.get_review_requests()

for user in users_requested:
    usernames_involved.add(user.username)

for team in teams_requested:
    for user in team.get_members():
        usernames_involved.add(user.username)

print(usernames_involved)

set()


# Implementation

In [1]:
import os
from numpy import *
import json
import pandas as pd
from random import *
import math
from datetime import *
from dateutil import parser
import itertools
from collections import defaultdict

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Load and Clean data

In [94]:
# Load data from drive
f = open('/content/drive/MyDrive/Colab Notebooks/My Project/openstack_final.json',)
data = json.load(f)  # returns json object as a dictionary
for i in range(5):   # 5 first elements of the json list
    print(data[i])
f.close()

{'time': '2011-07-18 15:43:34', 'changeId': 'Ibb2f047c9c45d011361d253d4444b2fd1ebd3612', 'ownerId': '1', 'reviewersId': ['2'], 'filePaths': ['modules/ssh/files/sshd_config', 'manifests/server.pp', 'modules/ssh/manifests/init.pp']}
{'time': '2011-07-18 16:56:22', 'changeId': 'Iea32ad3aedbee996db53655842061d14c9e3d876', 'ownerId': '1', 'reviewersId': ['1'], 'filePaths': ['modules/jenkins_slave/manifests/jenkinsuser.pp', 'modules/jenkins_slave/files/known_hosts']}
{'time': '2011-07-18 17:14:05', 'changeId': 'Ib6af08a6af71a6e5853d958a12f63c381eafa039', 'ownerId': '1', 'reviewersId': ['1'], 'filePaths': ['modules/jenkins_slave/manifests/jenkinsuser.pp']}
{'time': '2011-07-20 14:37:53', 'changeId': 'Ide7975c522b6b31a23af67b135135c07d75d9bde', 'ownerId': '2', 'reviewersId': ['1'], 'filePaths': ['modules/jenkins_slave/files/pubring.gpg', 'modules/jenkins_slave/manifests/jenkinsuser.pp']}
{'time': '2011-07-20 15:12:47', 'changeId': 'I28119ac1735127812866a4b0519368a829e2e666', 'ownerId': '2', 'r

In [95]:
# convert times to datetime format
for pr in data:
  pr['time'] = parser.parse(pr['time'])

data[0]

{'changeId': 'Ibb2f047c9c45d011361d253d4444b2fd1ebd3612',
 'filePaths': ['modules/ssh/files/sshd_config',
  'manifests/server.pp',
  'modules/ssh/manifests/init.pp'],
 'ownerId': '1',
 'reviewersId': ['2'],
 'time': datetime.datetime(2011, 7, 18, 15, 43, 34)}

In [96]:
len(data)

6545

In [97]:
# Save only PRs with 1 reviwer (to make the results similar to the paper)
final_data = []
for pr in data:
  if len(pr['reviewersId']) == 1:
    final_data.append(pr)

len(final_data)

3786

## Approach

In [233]:
# Split train/test sets
train = final_data[0:3400]
test = final_data[3400]

test

{'changeId': 'Idf77e2ed3d0cce61b01f3ca2ef8844beb65c77be',
 'filePaths': ['doc/src/docbkx/openstack-compute-admin/pom.xml',
  'doc/src/docbkx/openstack-compute-admin/computescheduler.xml',
  'doc/src/docbkx/openstack-compute-admin/computeconfigure.xml'],
 'ownerId': '3114',
 'reviewersId': ['964'],
 'time': datetime.datetime(2012, 4, 25, 15, 4, 30)}

In [100]:
def get_all_reviewers():
  # create a set of all reviewers
  all_reviewers = set()   
  for pr in final_data:
    for r in pr['reviewersId']:
      all_reviewers.add(r)
  return all_reviewers

In [101]:
all_reviewers = list(get_all_reviewers())
M = len(all_reviewers)

M

76

In [242]:
lambda_val = 7  # can be changed

### Time-decay Relationship

In [249]:
def find_PRlist(pull_reqs, reviewerId, cur_time):

  # lambda days before the time of our test PR
  lambda_ago = cur_time - timedelta(days=lambda_val)

  # list of PRs in "pull_reqs" that are between time of lambda days ago up to the time of test PR
  PS = [pr for pr in pull_reqs if ((lambda_ago < pr['time'] < cur_time) and (reviewerId in pr['reviewersId']))]
  # print("size of final list:", len(PS))

  # PS will be the list of PRs which submitted by the owner of test PR and reviewed by any reviewer in lambda days ago
  return PS 

In [235]:
def cal_decayRelation(pull_reqs, cur_time):

  # decayRelationship = 0.0
  # for pr in pull_reqs:
  #   diff_days = (cur_time-pr['time']).days
  #   # print(diff_days)
  #   if diff_days != 0:
  #     decayRelationship += 1/diff_days

  # OR
  decayRelationship = sum([(1/(cur_time-pr['time']).days) for pr in pull_reqs if ((cur_time-pr['time']).days!=0)])

  return decayRelationship

### Time-decay File Similarity

In [236]:
def pathDistance(f1, f2):
  return len(os.path.commonprefix([f1, f2]).split("/")[0:-1]) / max(len(f1.split("/")), len(f2.split("/")))

In [256]:
def prDistance(pr1, pr2):

  # path_dist_sum = 0
  # for f1 in pr1['filePaths']:
  #   for f2 in pr2['filePaths']:
  #    # print(pathDistance(f1, f2))
  #     path_dist_sum += pathDistance(f1, f2)

  # OR
  path_dist_sum = sum([pathDistance(f1, f2) for f1 in pr1['filePaths'] for f2 in pr2['filePaths']])
  
  return path_dist_sum/(len(pr1['filePaths'])*len(pr2['filePaths']))

In [261]:
def cal_decayFileSimilarity(pull_reqs, cur_time, test_pr):

  # decayFileSimilarity = 0.0
  # for pr in pull_reqs:
  #   diff_days = (cur_time-pr['time']).days
  #   # print(diff_days)
  #   if diff_days != 0:
  #     decayFileSimilarity += (prDistance(test_pr, pr) * (1/diff_days))

  # OR
  decayFileSimilarity = sum([prDistance(test_pr, pr)*(1/(cur_time-pr['time']).days) for pr in pull_reqs if ((cur_time-pr['time']).days!=0)])

  return decayFileSimilarity

### Main

In [263]:
# For each test PR:

# list of PRs that submitted by the owner of this PR in trainset
OS = [pr for pr in train if (pr['ownerId'] == test['ownerId'])]
# print("OS", len(OS))

cur_time = test['time']

# Calculating time-decaying Relationship for each reviewer in whole project and put these values to a vector
PRvector = [cal_decayRelation(find_PRlist(OS, rev, cur_time), cur_time) for rev in all_reviewers]

# Calculating time-decaying File Similarity
PRvector.extend([cal_decayFileSimilarity(find_PRlist(train, rev, cur_time), cur_time, test) for rev in all_reviewers])

print("Pull Request Vector:", PRvector)

Pull Request Vector: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.2399999999999998, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.09999999999999999, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.9740740330637743, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
